In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://www.instagram.com')
time.sleep(2)

# email='****@naver.com'
# input_id = driver.find_elements(By.CSS_SELECTOR,'input._aa4b')[0]
# input_id.clear()
# input_id.send_keys(email)
# 
# password='****'
# input_pw = driver.find_elements(By.CSS_SELECTOR,'input._aa4b')[1]
# input_pw.clear()
# input_pw.send_keys(password)
# input_pw.submit()
# time.sleep(30)

In [ ]:
import re
from bs4 import BeautifulSoup
import unicodedata
import pandas as pd

def insta_searching(word):
  url ='https://www.instagram.com/explore/tags/' + word
  return url

def select_first(driver):
  first = driver.find_element(By.CSS_SELECTOR,'div._aagw')
  first.click()
  time.sleep(5)

def get_content(driver):
  html = driver.page_source
  soup = BeautifulSoup(html,'lxml')

  try:
    content = soup.select('div._a9zs > h1')[0].text
    content = unicodedata.normalize('NFC',content)
  except:
    content=''

  tags = re.findall(r'#[^\s#,\\]+', content)
  # print('tags', tags)
  date = soup.select('time._a9ze._a9zf')[0]['datetime'][:10]
  # print('date', date)
  try:
    like = soup.select('a.x1i10hfl.xjbqb8w.x1ejq31n.xd10rxx.x1sy0etr.x17r0tee.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.x1ypdohk.xt0psk2.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x16tdsg8.x1hl2dhg.xggy1nq.x1a2a7pz.notranslate._a6hd > span')[0].text

    # print('like', like)
  except:
    like=''

  try:
    place = soup.select('div._ap3a > a.x1i10hfl')[0].text
    # print('place',place)
  except:
    place=''

  data = [content, date, like, place, tags]
  return data

def move_next(driver):
  try:
    right = driver.find_element(By.CSS_SELECTOR,'div._aaqg > button._abl-')
    right.click()
    time.sleep(3)
  except:
    print('no button')

word_list=[  '서울비건맛집','서울비건카페', '서울비건식당','서울비건레스토랑']
for word in word_list:
  url=insta_searching(word)
  driver.get(url)
  time.sleep(7)
  select_first(driver)
  results=[]
  target_cnt = 28
  for idx in range(target_cnt):
    try:
      data = get_content(driver)
      results.append(data)
      move_next(driver)
    except:
      time.sleep(2)
      move_next(driver)

  print(len(results))

  results_df = pd.DataFrame(results)
  results_df.columns=['content','data','like','place','tags']
  results_df.to_excel('./instagram_crawling_{}.xlsx'.format(word), index=False)

In [ ]:
seoul_vegan_insta_df=pd.DataFrame([])
word_list=[  '서울비건맛집',  '서울비건카페', '서울비건식당','서울비건레스토랑']
file_list=[]
for word in word_list:
  file_list.append('./instagram_crawling_{}.xlsx'.format(word))
  
for file in file_list:
  temp = pd.read_excel(file)
  seoul_vegan_insta_df = pd.concat([seoul_vegan_insta_df,temp])

seoul_vegan_insta_df.columns=['content','data','like','place','tags']
seoul_vegan_insta_df.drop_duplicates(subset=['content'], inplace=True)
seoul_vegan_insta_df.to_excel('./instagram_crawling_result-seoul-vegan-restaurant.xlsx', index=False)

In [ ]:
import pandas as pd
raw_total = pd.read_excel('./instagram_crawling_result-seoul-vegan-restaurant.xlsx')
tags_total=[]
for tags in raw_total['tags']:
  # print('tags',len(tags), tags)
  if len(tags) == 2:
    continue
  tags_list = tags[2:-2].split("', '")
  for tag in tags_list:
    # print('tag',tag)
    tags_total.append(tag)

In [ ]:
from collections import Counter
print('len',len(tags_total))
delete_word=['#존맛탱','#일러스트','#디엠']
clean_tag_total = [] 
for tag in tags_total:
  if tag not in delete_word:
    clean_tag_total.append(tag)
  
tag_count = Counter(tags_total)
clean_tag_total = Counter(clean_tag_total)
clean_tag_total.most_common(50)

In [ ]:
from hangul_font import to_hangul_font
import matplotlib.pyplot as plt
import seaborn as sns

to_hangul_font()
tag_count_df = pd.DataFrame(clean_tag_total.most_common(30))
tag_count_df.columns = ['tags','counts']
print(tag_count_df)
plt.figure(figsize=(10,8))
sns.barplot(x='counts',y='tags',data=tag_count_df)

In [ ]:
from wordcloud import WordCloud
import platform

if platform.system() == 'Windows':
  font_path = 'c:/Windows/Fonts/malgun.ttf'
elif platform.system() == 'Darwin':
  font_path = '/Users/$User/Library/Fonts/AppleGothic.ttf'
  
wordcloud=WordCloud(font_path=font_path,
                    background_color='white',
                    max_words=100,
                    relative_scaling=0.3,
                    width=800,
                    height=400
                    ).generate_from_frequencies(clean_tag_total)
plt.figure(figsize=(15,10))
plt.imshow(wordcloud)
plt.axis('off')
plt.savefig('./seoul_vegan_tag_wordcloud.png')


In [ ]:
import pandas as pd
import re
from collections import Counter

raw_total = pd.read_excel('./instagram_crawling_result-seoul-vegan-restaurant.xlsx')
clean_location_total = []
reg = re.compile('[0-9]')
clean_location_total=[]
for location in raw_total['place']:
  # print(reg.findall(location))
  if len(reg.findall(location)) == 0:
    clean_location_total.append(location)

clean_location_cnt = Counter(clean_location_total)
# print(clean_location_cnt.most_common(40))
location_count_df= pd.DataFrame(clean_location_cnt.most_common(40))
location_count_df.columns=['place','counts']
location_count_df.to_excel('./seoul_vegan_location_count.xlsx', index=False)


In [ ]:
locations = clean_location_cnt.keys()
locations

In [ ]:
import requests

def find_place(query):
  
  # 카카오 로컬 > 키워드로 검색하기 공식문서 https://developers.kakao.com/docs/latest/ko/local/dev-guide#search-by-keyword
  url='https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(query)
  headers={
    "Authorization":"KakaoAK e1ff570612513d98daff33c7868daa5d"
  }
  
  places = requests.get(url,headers=headers).json()['documents']
  place = places[0]
  name=place['place_name']
  x=place['x'] #경도
  y=place['y'] #위도
  
  return [name,x,y,query]


import time
location_info_list = []
for name in locations:
  try:
    data=find_place(name)
    location_info_list.append(data)
    time.sleep(0.5)
  except:
    pass

location_info_list_df = pd.DataFrame(location_info_list)
location_info_list_df.columns=['offical_name','latitude','longitude','place']
location_info_list_df.to_excel('./seoul_vegan_location_info_list.xlsx', index=False)


In [ ]:
location_count_df=pd.read_excel('./seoul_vegan_location_count.xlsx', index_col=0)
location_info_list_df=pd.read_excel('./seoul_vegan_location_info_list.xlsx')
location_data = pd.merge(location_info_list_df,location_count_df,how='inner', left_on='place',right_index=True)
location_data.head()

In [ ]:
location_data['place'].value_counts()

In [ ]:
location_data=location_data.pivot_table(
    index=['place','latitude','longitude'],
    values='counts',
    aggfunc='sum'
)
location_data.head()
location_data.to_excel('./seoul_vegan_location_pivot_data.xlsx')


In [ ]:
import pandas as pd

location_data = pd.read_excel('./seoul_vegan_location_pivot_data.xlsx')

In [ ]:
import folium

namsan=[37.55120584342893, 126.98346716392766]
seoul_map = folium.Map(location=namsan,zoom_start=11)

for idx in range(len(location_data)):
  name = location_data['place'][idx]
  cnt = location_data['counts'][idx]
  size=int(cnt)*2
  long = float(location_data['longitude'][idx]) #경도
  lat = float(location_data['latitude'][idx]) #위도
  print(name, cnt, long, lat)
  folium.CircleMarker((lat,long), radius=size, color='red',popup=name).add_to(seoul_map)
seoul_map.save('./seoul_map_new.html')

In [ ]:
from folium.plugins import MarkerCluster

locations=[]
names=[]

for idx in range(len(location_data)):
  data = location_data.iloc[idx]
  # print(data)
  locations.append((float(data['latitude']),float(data['longitude'])))
  names.append(data['place'])

namsan=[37.55120584342893, 126.98346716392766]
seoul_map2 = folium.Map(location=namsan, zoom_start=11)
marker_cluster=MarkerCluster(
    locations=locations,
    popups=names,
    name='Seoul',
    overlay=True,
    control=True
)

# print(names)
# print(locations)

marker_cluster.add_to(seoul_map2)
folium.LayerControl().add_to(seoul_map2)
seoul_map2.save('./seoul_map2.html')


In [ ]:
import pandas as pd
raw_total = pd.read_excel('./instagram_crawling_result-seoul-vegan-restaurant.xlsx')
select_word='망원'
check_list=[]
for content in raw_total['content']:
  if select_word in content:
    check_list.append(True)
  else:
    check_list.append(False)
    
select_df = raw_total[check_list]

for idx in select_df.index:
  print(select_df.loc[idx, 'content'])
  print('-'*50)
select_df.head()